In [2]:
import os
import boto3
import joblib
import shutil
import mido
import miditoolkit
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim.downloader

from ast import literal_eval
from pathlib import Path
from tqdm.notebook import tqdm

import dotenv
dotenv.load_dotenv()

sns.set()

In [3]:
# set some parameter
DATA_DIR = '../dataset'

In [4]:
# read in master metadata
metamidi = pd.read_csv(os.path.join(DATA_DIR, 'master_midi_meta_parse.csv'))
metamidi

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
0,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001,NaN,"['Am', 'G', 'Dm', 'C', 'D']",4,string,classical,0
1,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002,NaN,"['Am', 'Dm', 'G', 'C', 'G7', 'F']",0,keyboard,classical,0
2,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003,NaN,"['F', 'E+', 'E', 'Am']",4,string,classical,0
3,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004,NaN,"['A#', 'Dm', 'G', 'C', 'F', 'Gm']",7,choir,classical,0
4,aminor,"[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005,NaN,"['Em', 'F', 'Am']",0,keyboard,classical,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72376,amajor,"[['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',...",unknown,12,169,soul,unknown,electric_guitar_jazz,unknown,4/4,70,79,val,e34a552e87b6bb8939bc35660d2d116c-5,unknown,"['G', 'A', 'Gm']",0,keyboard,soul,11
72377,gminor,"[['Gm', 'Gm', 'Gm', 'Gm', 'B', 'B', 'B', 'B', ...",unknown,3,100,oldie,unknown,soprano_sax,unknown,4/4,116,116,val,d0531034be492842b6d9bb32486f6a95-1,unknown,"['B', 'Gm']",5,brass,oldie,8
72378,emajor,"[['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',...",unknown,11,180,rock,unknown,bright_acoustic_piano,unknown,2/4,95,95,val,72b9d074542398fc021f636eb17da32b-5,unknown,['B'],0,keyboard,rock,10
72379,c#major,"[['G#', 'G#', 'G#', 'G#', 'C#', 'C#', 'C#', 'C...",unknown,11,240,metal,unknown,acoustic_grand_piano,unknown,4/4,95,95,val,0bc159de1686720e1c0282b57a3af7e5-4,unknown,"['C#', 'G#', 'E']",0,keyboard,metal,7


# Applying Cosine Similarity to Group Instruments